### 최적의 파라미터를 확인
- pipe_multi에서 최적의 파라미터를 생성
- step 1
    - vector화 작업
        - ngram_range = (1, 1), (1, 2)
- step 2
    - LinearSVC를 병렬 작업
        - C = 1.0, 1.5, 2.0
        - class_weight = None, 'balanced'
- 교차 검증의 횟수는 3회
- 평가 점수는 정확도
- 최적의 매개변수 값 확인

In [80]:
import os 
from glob import glob
import pandas as pd

In [81]:
# 특정 경로의 파일의 목록을 가져오는 기능 
# os 라이브러리를 이용
os.listdir('./')

['01_review.ipynb',
 '1-1.여성의류(196).json',
 '1-1.여성의류(197).json',
 '1-1.여성의류(198).json',
 '1-1.여성의류(199).json',
 '1-1.여성의류(200).json',
 '1-1.여성의류(201).json',
 '1-1.여성의류(202).json',
 '1-1.여성의류(203).json',
 '1-1.여성의류(204).json',
 '1-1.여성의류(205).json',
 '1-1.여성의류(206).json',
 '1-1.여성의류(207).json',
 '1-1.여성의류(208).json',
 '1-1.여성의류(209).json']

In [82]:
# glob 이용 
# 장점 : 파일의 경로와 파일의 이름을 하나의 리스트로 생성 
#       특정 확장자만 선택해서 리스트로 생성이 가능
json_list = glob("./*.json")
json_list

['.\\1-1.여성의류(196).json',
 '.\\1-1.여성의류(197).json',
 '.\\1-1.여성의류(198).json',
 '.\\1-1.여성의류(199).json',
 '.\\1-1.여성의류(200).json',
 '.\\1-1.여성의류(201).json',
 '.\\1-1.여성의류(202).json',
 '.\\1-1.여성의류(203).json',
 '.\\1-1.여성의류(204).json',
 '.\\1-1.여성의류(205).json',
 '.\\1-1.여성의류(206).json',
 '.\\1-1.여성의류(207).json',
 '.\\1-1.여성의류(208).json',
 '.\\1-1.여성의류(209).json']

In [83]:
# json_list를 이용하여 하나의 데이터프레임으로 단순 행 결합(concat)

# 빈 데이터프레임을 생성
total_df = pd.DataFrame()

for file_path in json_list[:5]:
    # print(file_path)
    df = pd.read_json(file_path)
    # print(df)
    # break
    # concat: +, concat 결과를 total_df에 대입: + =
    total_df = pd.concat( [total_df, df], axis= 0)

total_df.reset_index(drop=True, inplace=True)

In [84]:
# 또는
# total_df = pd.concat( [pd.read_json(file_path) for file_path in json_list[:5] ]).info()

In [85]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523 entries, 0 to 522
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            523 non-null    int64  
 1   RawText          523 non-null    object 
 2   Source           523 non-null    object 
 3   Domain           523 non-null    object 
 4   MainCategory     523 non-null    object 
 5   ProductName      523 non-null    object 
 6   Syllable         523 non-null    int64  
 7   Word             523 non-null    int64  
 8   GeneralPolarity  519 non-null    float64
 9   Aspects          523 non-null    object 
dtypes: float64(1), int64(3), object(6)
memory usage: 41.0+ KB


In [86]:
# Aspects의 데이터를 하나로 합치고 새로운 데이터 프레임을 생성
df_aspects = pd.DataFrame(sum(total_df['Aspects'], []))

In [87]:
df_aspects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3963 entries, 0 to 3962
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Aspect             3963 non-null   object
 1   SentimentText      3963 non-null   object
 2   SentimentWord      3963 non-null   object
 3   SentimentPolarity  3963 non-null   object
dtypes: object(4)
memory usage: 124.0+ KB


In [88]:
# 데이터의 불균형 문제 확인
df_aspects['SentimentPolarity'].value_counts()

SentimentPolarity
1     3353
-1     479
0      131
Name: count, dtype: int64

In [89]:
# 결측치 존재 여부 확인
df_aspects.isna().sum()

Aspect               0
SentimentText        0
SentimentWord        0
SentimentPolarity    0
dtype: int64

In [90]:
# 데이터셋에서 문자열의 좌우의 공백을 제거 
# 모든 컬럼이 Object 형이기 때문에 strip() 바로 사용 가능
df_aspects = df_aspects.map(lambda x : x.strip())

In [91]:
# 빈 텍스트 존재 여부 확인
df_aspects.isin( [''] ).sum()

Aspect               0
SentimentText        0
SentimentWord        0
SentimentPolarity    0
dtype: int64

In [92]:
df_aspects['SentimentText'].value_counts()

SentimentText
가볍고                                18
따뜻하고                               14
저렴한 가격에                             8
좋은 가격에                              6
따뜻합니다.                              4
                                   ..
소재가 구김도 없어 좋고                       1
엄청 가벼워서 입은 것 같지도 않은 느낌이 아주 좋아요.     1
입었을 때 포근하게 감싸는 느낌도 엄청 만족스럽네요.       1
일단 보는 순간 고급스럽다는 생각이 탁 드는 소재였습니다     1
모임때나 결혼식이나 출근복으로 좋은 이 원피스           1
Name: count, Length: 3853, dtype: int64

In [93]:
before_cnt = len(df_aspects)

df_aspects.drop_duplicates('SentimentText', inplace=True)

after_cnt = len(df_aspects)

print(f"제거가 된 행의 개수 {before_cnt - after_cnt}")

제거가 된 행의 개수 110


In [94]:
# 1, 0, -1 의 비율을 확인 
df_aspects['SentimentPolarity'].value_counts()

SentimentPolarity
1     3247
-1     477
0      129
Name: count, dtype: int64

In [95]:
# 인덱스 초기화
df_aspects.reset_index(drop=True, inplace=True)

In [96]:
# 토큰화 -> 벡터화
from konlpy.tag import Komoran
from sklearn.feature_extraction.text import TfidfVectorizer

komoran = Komoran()
allow_pos = ['NNP','NNG','VV','VA','MAG','SL']

def komoran_tokenize(text):
    tokens = []
    for word, pos in komoran.pos(text):
        if (pos in allow_pos) & (len(word) >= 2):
            tokens.append(word)
    return tokens

vectorizer = TfidfVectorizer(
    tokenizer= komoran_tokenize,
    ngram_range= (1, 2),
    min_df= 3,
    max_df= 0.8,
    max_features= 30000
)


In [97]:
# 모델 생성 
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

In [98]:
svc = LinearSVC(random_state=42, class_weight='balanced')

multi_model = MultiOutputClassifier(svc)

pipe = Pipeline(
    [
        ('vector', vectorizer), 
        ('model', multi_model)
    ]
)

In [99]:
# 계층화 폴드 
from sklearn.model_selection import KFold

skfold = KFold(n_splits=3, shuffle= True, 
                         random_state=42)

In [100]:
df_aspects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3853 entries, 0 to 3852
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Aspect             3853 non-null   object
 1   SentimentText      3853 non-null   object
 2   SentimentWord      3853 non-null   object
 3   SentimentPolarity  3853 non-null   object
dtypes: object(4)
memory usage: 120.5+ KB


In [101]:
from sklearn.preprocessing import LabelEncoder

In [102]:
le = LabelEncoder()
df_aspects['Aspect'] = le.fit_transform(df_aspects['Aspect'])
df_aspects['SentimentPolarity'] = df_aspects[
    'SentimentPolarity'].astype('int')

In [103]:
# 독립 변수 , 종속 변수 생성
X = df_aspects['SentimentText'].values
Y = df_aspects[['Aspect', 'SentimentPolarity']].values

In [104]:
print(X.shape, Y.shape)

(3853,) (3853, 2)


In [105]:
print(type(Y[0][0]), type(Y[0][1]))

<class 'numpy.int64'> <class 'numpy.int64'>


In [106]:
from sklearn.model_selection import GridSearchCV

In [107]:
params = {
    'model__estimator__C' : [1.0, 2.0]
}
grid = GridSearchCV(
    estimator=pipe, 
    param_grid= params, 
    cv = skfold, 
    scoring="accuracy"
)

In [108]:
import warnings
warnings.filterwarnings('ignore')

In [109]:
grid.fit(X, Y)

,estimator,Pipeline(step..._state=42)))])
,param_grid,"{'model__estimator__C': [1.0, 2.0]}"
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [110]:
grid.best_estimator_

,steps,"[('vector', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,<function kom...00200C27B4220>


---
#### 연습
1. total_df 에서 RawText 컬럼의 데이터들과 Kkma를 이용하여 문장 별로 나눠준다.
2. grid의 best_estimator_에서 예측 실행
3. 실행된 결괏값을 이용하여 데이터프레임(rawText, Aspect_pred, Pola_pred)으로 생성
4. RawText, Aspect_pred 값을 이용하여 그룹화 -> 그룹화 연산에는 평균 사용

In [111]:
best_model = grid.best_estimator_

In [112]:
total_df.columns

Index(['Index', 'RawText', 'Source', 'Domain', 'MainCategory', 'ProductName',
       'Syllable', 'Word', 'GeneralPolarity', 'Aspects'],
      dtype='object')

In [113]:
total_df.loc[0, 'RawText']

'안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 딱 좋을 인생 경량 패딩 하나 소개해 드릴게요.  다 함께 go go go~~  제가 입어보고 좋아서 엄마께도 구매해드렸네요. 딱 기본 스타일인데 또 입은 거 보면 깔끔하게 저렴해보이지 않는 디자인이라서 어디서 샀냐고 많이들 물어보는 옷입니다.  엄마도 제가 입은 것 보시더니 탐내셔서  주문해드렸어요. 영하로 내려가는 날씨에는 이것만 입기엔 얇지만 초겨울까지는 운동 갈때 안에 얇은 기능성 반팔 입고 요것만 입어도 꽤 따뜻해요. 색상도 디자인도 무난해서 사무실에 두고 입기도 좋고, 평소에 가볍게 나갈때 코디하기도 좋아요. 앞으로 코트를 입거나 할때 안에 내피로 활용하기도 딱일 듯요! 맘같아선 색깔별로 쟁이고 싶습니다.  20대와 50대 모두 아우르는 디자인이 무난하고 깔끔하면서 고급스러운 옷 추천합니다~~~  설명 끝! 좀 도움이 되셨나요? 그러면 좋아요 꾹 눌러 주시고 전 이만 총총총~~ 항상 여러 이웃님들께 감사드립니다. '

In [114]:
# 문단인 장문의 데이터에서 문장별로 나눠주기 
from konlpy.tag import Kkma

In [115]:
# --- (1) Kkma를 이용하여 RawText를 문장별로 분할 ---
kkma = Kkma()

# raw_list - 리뷰 문단을 문장으로 나눈 리스트를 담기 위한 공간
raw_list = []
# raw_dict - 리뷰 문단마다 index는 키 값, value는 리뷰 문단
raw_dict = {}

# Kkma의 sentenses() 메소드를 사용하여 문장 분리
for i in range(len(total_df)):
    # print(kkma.sentences(total_df.loc[i, 'RawText']))
    # break
    raw_list.append(kkma.sentences(total_df.loc[i, 'RawText']))
    raw_dict[i] = total_df.loc[i, 'RawText']

In [116]:
sentence_df = pd.DataFrame()
for idx, raw in enumerate(raw_list):
    # print(raw)
    # 종속이 2개니까 예측값이 2차원
    pred = best_model.predict(raw)
    # print(pred)
    # 예측값들을 데이터프레임으로 만들어 컬럼명을 Aspect_pred 로 지정
    temp_df = pd.DataFrame(pred, columns = ['Aspect_pred', 'Pola_pred'])
    # idx번째에 있는 실제 리뷰데이터들을 가져옴
    temp_df['RawText'] = raw_dict[idx]
    # display를 사용하면 보기 쉬운 표 형태로 출력됨 - 확인용
    # display(temp_df)
    # 단순 행 결합 - 각 리뷰마다 문장별로 나온 예측값을 결합
    sentence_df = pd.concat([sentence_df, temp_df])
    # break

In [117]:
sentence_df

,Aspect_pred,Pola_pred,RawText
0,1,1,안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 ...
1,1,1,안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 ...
2,1,1,안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 ...
3,1,1,안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 ...
4,5,0,안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 ...
...,...,...,...
5,9,1,날씨가 점점 쌀쌀한 것 같아요!! 이런 날씨에 입기 좋은 경량 패딩을 소개 해드리고...
6,16,1,날씨가 점점 쌀쌀한 것 같아요!! 이런 날씨에 입기 좋은 경량 패딩을 소개 해드리고...
7,1,1,날씨가 점점 쌀쌀한 것 같아요!! 이런 날씨에 입기 좋은 경량 패딩을 소개 해드리고...
8,5,1,날씨가 점점 쌀쌀한 것 같아요!! 이런 날씨에 입기 좋은 경량 패딩을 소개 해드리고...


In [118]:
# aspect_pred을 기준으로 리뷰 전체를 그룹화
group_df = sentence_df.groupby(['RawText', 'Aspect_pred']).mean()  # 어차피 컬럼 하나밖에 안 남았으니 mean

In [119]:
# total_df 에 넣기 위해 인덱스 리셋
group_df.reset_index(inplace=True)  # 기존 인덱스 없애면(drop_True) pola_pred의 인덱스만 남아 의미 없어짐

In [120]:
# 원본으로 교체해 다시 'Aspect_pred'에 넣어줌
group_df['Aspect_pred'] = le.inverse_transform(group_df['Aspect_pred'])

In [121]:
# 참고) 데이터 로드 후 columns 확인 필수!!!

# axis= 0일 땐 밑으로 합칠거니까 테이블(컬럼)의 구조가 같아야 함.
# axis= 1일 땐 인덱스가 같아야 함
# 컬럼의 이름이 모두 같은 걸 아니까 단순 행 결합했던 것.
# total_df와 group_df 컬럼의 구조가 같은가? 
# -> columns 로 확인해보니 다름!
group_df.columns
total_df.columns

# 일반적으로 단순 열 결합은 잘 사용하지 않음.
# total_df와 group_df 인덱스가 같은가?
# -> index 로 확인해보니 다름!
group_df.index
total_df.index

RangeIndex(start=0, stop=523, step=1)

In [122]:
# total_df와 group_df 조인 결합
review_df = pd.merge(total_df, group_df, on= 'RawText', how= 'inner')
# -> unique()를 통해 shape 사이즈로 집합의 크기 확인
# -> 두 집합의 인덱스 차이는 큰데, 유니크한 값의 개수는 같음
# -> total_df에서 group_df의 데이터를 가져온 것이므로 같은 집합 두 개가 겹쳐 있는 형태
# -> how에 어떤 것을 지정해도 같은 값이 나오므로 뭘 해도 상관 없음

**merge( )**
</br>: df 2개를 join 결합
- 매개변수
    - on - 공통적으로 가진 데이터 -> head()를 통해 확인
    - how - 어떻게 합칠 것인가? 교집합? 합집합? 왼쪽? 오른쪽?
        - inner(교집합. 결측치가 가장 안 나오는 결합 방식)
        - outer(합집합. 데이터를 많이 살릴 수 있지만 결측치가 가장 많이 나오는 결합 방식)
        - left(집합 A)
        - right(집합 B)

In [123]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3136 entries, 0 to 3135
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            3136 non-null   int64  
 1   RawText          3136 non-null   object 
 2   Source           3136 non-null   object 
 3   Domain           3136 non-null   object 
 4   MainCategory     3136 non-null   object 
 5   ProductName      3136 non-null   object 
 6   Syllable         3136 non-null   int64  
 7   Word             3136 non-null   int64  
 8   GeneralPolarity  3105 non-null   float64
 9   Aspects          3136 non-null   object 
 10  Aspect_pred      3136 non-null   object 
 11  Pola_pred        3136 non-null   float64
dtypes: float64(2), int64(3), object(7)
memory usage: 294.1+ KB


In [124]:
# ProductName의 빈도수 체크
# value_counts, unique, ...
len(review_df['ProductName'].unique())

214

In [125]:
# 제품별 리뷰의 상세 감정 분석 가능
# 제품 이름 중 가장 많은 리뷰를 가진 제품을 선택하여 감정 점수의 평균 확인
review_df.groupby(['ProductName', 'RawText']).n()

AttributeError: 'DataFrameGroupBy' object has no attribute 'n'

In [ ]:
pd.pivot_table(
    data = review_df.drop_duplicates('RawText'),
    index = 'ProductName', 
    values = 'RawText', 
    aggfunc= ('count')
).sort_values('RawText',ascending=False)

In [ ]:
product_name = 'OO 코튼 가디건'
# 해당 제품의 리뷰들의 전체적인 감정 점수 출력

# case1
# ProductName에서 필터링을 한 뒤 Aspect_pred를 기준으로 그룹화 -> Pola_pred의 평균
test_df = review_df.loc[review_df['ProductName'] == product_name, ]
test_df

In [ ]:
test_df.groupby('Aspect_pred')['Pola_pred'].mean()

In [ ]:
# case 2
# case 1은 매번 계산해야 하지만, case 2는 저장시켜두고 찾아서 사용할 수 있으므로 강사님은 개인적으로 case 2 추천!
# review_df에서 Product_name과 Aspect_pred를 기준으로 그룹화 -> Pola_pred의 평균
# 원하는 제품명을 선택해 확인
group_df2 = review_df.groupby(['ProductName', 'Aspect_pred'])['Pola_pred'].mean()

In [ ]:
# 결과는 같은데 순서만 바뀜
group_df2[product_name]